## Two stage flair training

In [0]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git@63aeabf9a18bdf53af3bcba5bd80f43ac717656e

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-ccur1r0t
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-ccur1r0t
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 665kB 6.1MB/s 
     |████████████████████████████████| 983kB 17.7MB/s 
     |████████████████████████████████| 256kB 37.5MB/s 
     |████████████████████████████████| 798kB 16.2MB/s 
     |████████████████████████████████| 3.8MB 39.8MB/s 
     |████████████████████████████████| 890kB 35.5MB/s 
     |████████████████████████████████| 1.1MB 41.5MB/s 
  Created wheel for flair: filename=flair-0.5-cp36-none-any.whl size=149375 sha256=b1eee8c2a38333738eb30bd3d268d8ce07f799e658608d38a395dcd303156b63
  Stored in directory: /tmp/pip-ephem-wheel-cache-9k8i2ivi/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Create

In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
from flair.data import Sentence
import pandas as pd
import numpy as np

from torch.optim.adam import Adam

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_folder = "./drive/My Drive/capstone/data/"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv(data_folder + "combined_benchmark.csv")

In [0]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
benchmark = benchmark.sample(frac=1)
benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

#### Build corpus

In [0]:
# corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
)

2020-05-21 21:33:57,559 Reading data from drive/My Drive/capstone/data
2020-05-21 21:33:57,560 Train: drive/My Drive/capstone/data/train.csv
2020-05-21 21:33:57,561 Dev: drive/My Drive/capstone/data/dev.csv
2020-05-21 21:33:57,563 Test: drive/My Drive/capstone/data/test.csv


#### Create word embeddings

In [0]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.



2020-05-21 21:34:20,508 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpx661a1f1


100%|██████████| 19689779/19689779 [00:00<00:00, 49444260.47B/s]

2020-05-21 21:34:21,096 copying /tmp/tmpx661a1f1 to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt


2020-05-21 21:34:21,125 removing temp file /tmp/tmpx661a1f1
2020-05-21 21:34:37,776 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmp119mrbt1


100%|██████████| 19689779/19689779 [00:00<00:00, 51132721.58B/s]

2020-05-21 21:34:38,322 copying /tmp/tmp119mrbt1 to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt


2020-05-21 21:34:38,354 removing temp file /tmp/tmp119mrbt1


#### First Stage Fine-tuning

In [0]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder, max_epochs=10)

2020-05-21 21:34:44,497 Computing label dictionary. Progress:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.
100%|██████████| 1314/1314 [00:02<00:00, 642.58it/s]

2020-05-21 21:34:46,855 [b'0', b'-1', b'1']
2020-05-21 21:34:46,874 ----------------------------------------------------------------------------------------------------
2020-05-21 21:34:46,877 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=Tru

2020-05-21 21:34:49,614 epoch 1 - iter 3/37 - loss 1.37792166 - samples/sec: 41.88
2020-05-21 21:35:04,758 epoch 1 - iter 6/37 - loss 1.38339380 - samples/sec: 50.24
2020-05-21 21:35:20,032 epoch 1 - iter 9/37 - loss 1.25956689 - samples/sec: 47.84
2020-05-21 21:35:35,205 epoch 1 - iter 12/37 - loss 1.27234081 - samples/sec: 44.19
2020-05-21 21:35:50,097 epoch 1 - iter 15/37 - loss 1.23371030 - samples/sec: 56.95
2020-05-21 21:36:05,057 epoch 1 - iter 18/37 - loss 1.22769420 - samples/sec: 57.27
2020-05-21 21:36:19,920 epoch 1 - iter 21/37 - loss 1.21607478 - samples/sec: 57.71
2020-05-21 21:36:35,063 epoch 1 - iter 24/37 - loss 1.20891273 - samples/sec: 52.59
2020-05-21 21:36:50,015 epoch 1 - iter 27/37 - loss 1.19474173 - samples/sec: 55.33
2020-05-21 21:37:04,955 epoch 1 - iter 30/37 - loss 1.17071977 - samples/sec: 52.95
2020-05-21 21:37:19,909 epoch 1 - iter 33/37 - loss 1.14583929 - samples/sec: 54.74
2020-05-21 21:37:34,727 epoch 1 - iter 36/37 - loss 1.15452903 - samples/sec: 5

{'dev_loss_history': [0.9445202946662903,
  0.931380569934845,
  1.0202172994613647,
  0.8029612898826599,
  0.8560692667961121,
  0.8320075869560242,
  1.0983926057815552,
  0.8370154500007629,
  1.3560564517974854,
  0.8271321654319763],
 'dev_score_history': [0.7006802721088435,
  0.6870748299319728,
  0.6961451247165533,
  0.7414965986394558,
  0.7596371882086168,
  0.7324263038548753,
  0.7052154195011338,
  0.746031746031746,
  0.6870748299319728,
  0.7732426303854876],
 'test_score': 0.776255707762557,
 'train_loss_history': [1.1594745307355314,
  0.9738980918317228,
  0.8259133777102908,
  0.7832235951681394,
  0.7250221081682153,
  0.703832321070336,
  0.6765379398255735,
  0.5982233124810297,
  0.632674384761501,
  0.4617382706822576]}

#### Data folder after first stage fine tuning
![alt text](https://raw.github.ubc.ca/ltian05/better_dwelling_capstone/aarontian/week_4/screenshots/Screen%20Shot%202020-05-25%20at%204.34.08%20PM.png?token=AAAAOMWRKXQXADFFGZWV72S62WILA) 

### Second Stage Employment (fine tune on hand annotated datasets)

In [0]:
new_data_folder = './drive/My Drive/capstone/data/second_stage_employment/'
new_column_name_map = {3: "text", 5: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, 
                                         delimiter=',',    # comma separated rows
)

2020-05-28 21:59:51,972 Reading data from drive/My Drive/capstone/data/second_stage_employment
2020-05-28 21:59:51,974 Train: drive/My Drive/capstone/data/second_stage_employment/train.csv
2020-05-28 21:59:51,977 Dev: drive/My Drive/capstone/data/second_stage_employment/dev.csv
2020-05-28 21:59:51,984 Test: drive/My Drive/capstone/data/second_stage_employment/test.csv


In [0]:
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-28 21:59:55,066 loading file ./drive/My Drive/capstone/data/best-model.pt


In [0]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-28 22:00:05,361 ----------------------------------------------------------------------------------------------------
2020-05-28 22:00:05,369 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [0.926588773727417,
  1.0328346490859985,
  0.8461851477622986,
  1.1260348558425903,
  1.4034475088119507,
  0.9472813010215759,
  1.0166206359863281,
  1.0762646198272705,
  1.0556421279907227,
  1.092240810394287],
 'dev_score_history': [0.7575757575757576,
  0.6363636363636364,
  0.6363636363636364,
  0.6363636363636364,
  0.5757575757575758,
  0.696969696969697,
  0.6363636363636364,
  0.5757575757575758,
  0.5757575757575758,
  0.5757575757575758],
 'test_score': 0.7575757575757576,
 'train_loss_history': [1.6566001176834106,
  0.9805337488651276,
  1.0864239037036896,
  0.9947719871997833,
  1.026974231004715,
  0.5524228066205978,
  0.4095269739627838,
  0.3753960132598877,
  0.32510678470134735,
  0.30383560061454773]}

### Employment folder after second stage fine tuning
![alt text](https://raw.github.ubc.ca/ltian05/better_dwelling_capstone/aarontian/week_4/screenshots/Screen%20Shot%202020-05-25%20at%204.24.58%20PM.png?token=AAAAOMRAGJI4MX5IUK52VM262WISU)

### Second Stage Stock Market (fine tune on hand annotated datasets)

In [0]:
new_data_folder = './drive/My Drive/capstone/data/second_stage_stock/'
new_column_name_map = {3: "text", 5: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, 
                                         delimiter=',',    # comma separated rows
)

2020-05-28 16:31:31,081 Reading data from drive/My Drive/capstone/data/second_stage_stock
2020-05-28 16:31:31,083 Train: drive/My Drive/capstone/data/second_stage_stock/train.csv
2020-05-28 16:31:31,086 Dev: drive/My Drive/capstone/data/second_stage_stock/dev.csv
2020-05-28 16:31:31,090 Test: drive/My Drive/capstone/data/second_stage_stock/test.csv


In [0]:
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-28 16:31:35,793 loading file ./drive/My Drive/capstone/data/best-model.pt


In [0]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, 
              max_epochs=10)

2020-05-28 16:32:07,516 ----------------------------------------------------------------------------------------------------
2020-05-28 16:32:07,521 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [1.2610609531402588,
  0.9575487971305847,
  1.0690481662750244,
  0.9532806277275085,
  0.7961318492889404,
  0.8032405972480774,
  0.9901302456855774,
  1.0061790943145752,
  1.409272313117981,
  1.34184730052948],
 'dev_score_history': [0.7222222222222222,
  0.6666666666666666,
  0.7222222222222222,
  0.7777777777777778,
  0.6666666666666666,
  0.8333333333333334,
  0.8333333333333334,
  0.8333333333333334,
  0.7222222222222222,
  0.7222222222222222],
 'test_score': 0.8333333333333334,
 'train_loss_history': [1.6039255857467651,
  0.9995986819267273,
  0.7806560397148132,
  1.0005999505519867,
  1.0892212986946106,
  0.6222394704818726,
  0.41739916801452637,
  0.36568620800971985,
  0.3373175859451294,
  0.5574379116296768]}

#### Stock folder after second stage fine tuning
![alt text](https://raw.github.ubc.ca/ltian05/better_dwelling_capstone/aarontian/week_4/screenshots/Screen%20Shot%202020-05-25%20at%204.28.07%20PM.png?token=AAAAOMRZQTYPKV4SLOVBE2K62WIXG)

### Use model to predict

In [0]:
def finetuned_model_predictions(input_file_path, finetuned_classifier, output_file_path):
  '''Makes Sentiment Predictions on unannotated data points contained in the input csvfile by loading the user-defined classifier.
     Exports the csvfile by adding two columns 'pred_label' and 'confidence' and filling in results from model predictions.
  '''

  unannotated_df = pd.read_csv(input_file_path)
  ## add two new columns to export predictions
  #unannotated_df['pred_label'] = None
  #unannotated_df['confidence'] = None
  unannotated_df['best_label'] = None
  unannotated_df['best_confidence'] = None
  unannotated_df['second_likely'] = None
  unannotated_df['second_confidence'] = None
  unannotated_df['least_likely'] = None
  unannotated_df['least_confidence'] = None


  for i in range(len(unannotated_df)):

    #print(unannotated_df['title_desc'].iloc[i])
    sentence = Sentence(unannotated_df['title_desc'].iloc[i])

    finetuned_classifier.predict(sentence,  multi_class_prob=True)

    pred_score_label = [(sentence.labels[c].score, sentence.labels[c].value) for c in range(len(sentence.labels))]
    pred_score_label.sort()

    # list in ascending order on confidence score
    best_label = int(pred_score_label[-1][1])
    best_confidence = pred_score_label[-1][0]
    second_likely_label = int(pred_score_label[-2][1]) 
    second_likely_confidence = pred_score_label[-2][0]
    least_likely_label = int(pred_score_label[0][1]) 
    least_likely_confidence = pred_score_label[0][0]

    #best_label_ind = np.argmax(pred_confs)
    #best_confidence = sentence.labels[best_label_ind].value
    #second_likely_ind = np.argsort(pred_confs)[-2] # array in ascending order
    #second_likely_confidence = np.sort(pred_confs)[-2]
    #least_likely_ind = np.argsort(pred_confs)[0]
    #least_likely_confidence = np.sort(pred_confs)[0]
    #print(sentence.labels)
    #print(sentence.labels[0].value)
    #print(sentence.labels[0].score)

    #label_dict = {0:1,1:-1,2:0}

    unannotated_df['best_label'].iloc[i] = best_label
    unannotated_df['best_confidence'].iloc[i] = best_confidence
    #print(label_dict[best_label_ind],best_confidence)
    unannotated_df['second_likely'].iloc[i] = second_likely_label
    unannotated_df['second_confidence'].iloc[i] = second_likely_confidence
    unannotated_df['least_likely'].iloc[i] = least_likely_label
    unannotated_df['least_confidence'].iloc[i] = least_likely_confidence

    #unannotated_df['pred_label'].iloc[i] = sentence.labels[0].value
    #unannotated_df['confidence'].iloc[i] = sentence.labels[0].score

  print(f"All { len(unannotated_df) } rows done prediction! ")

  unannotated_df.to_csv(output_file_path,index=False)

  print("Done export!")


#use case
#new_data_folder = '/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/'
#finetuned_classifier = TextClassifier.load(new_data_folder + 'best-model.pt')
#input_file_path = '/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/unannotated_for_predictions/predictions_dataset_GDP_Bloomberg.csv'
#output_file_path = '/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/unannotated_for_predictions/unannotated_GDP_Bloomberg_predictions.csv'

#finetuned_model_predictions(input_file_path, finetuned_classifier, output_file_path)

In [0]:
prediction_file = "./drive/My Drive/capstone/data/prediction/"
bloomberg_employment = pd.read_csv(prediction_file + 'predictions_dataset_employment_Bloomberg.csv')
for col, row in bloomberg_employment.iterrows():
  title_desc = row['title_desc']
  publish_date = row['publishedAt']
  predicted = employment_classifier.predict(title_desc, multi_class_prob=True)
  print(predicted)
  pred_list = [(predicted[0].labels[c].score, predicted[0].labels[c].value) for c in range(len(predicted[0].labels))]
  print(pred_list)
  pred_list.sort()
  print(pred_list)
  
  #print(predicted[0].labels[0].value)
  #print(predicted[0].labels[0].score)
  #print(predicted[0].labels[1])
  break



[Sentence: "Goldman Sachs now sees U.S. jobless rate peaking at 25%, not 15%. Goldman Sachs Group Inc. economists revised their forecasts to reflect a gloomier outlook for the U.S. labor market, though also the potential for a faster recovery from the coronavirus pandemic."   [− Tokens: 42  − Sentence-Labels: {'class': [0 (0.0043), -1 (0.8335), 1 (0.1621)]}]]
[(0.004326847847551107, '0'), (0.8335288763046265, '-1'), (0.1621442437171936, '1')]
[(0.004326847847551107, '0'), (0.1621442437171936, '1'), (0.8335288763046265, '-1')]


In [0]:
employment_data_folder = './drive/My Drive/capstone/data/second_stage_employment/'
prediction_file = "./drive/My Drive/capstone/data/prediction/"
employment_file_path = prediction_file + 'predictions_dataset_employment_Bloomberg.csv'
employment_classifier = TextClassifier.load(employment_data_folder + 'best-model.pt')
employment_out_file_path = 'unannotated_employment_Bloomberg_predictions.csv'

2020-05-28 23:43:16,629 loading file ./drive/My Drive/capstone/data/second_stage_employment/best-model.pt


In [0]:
stock_data_folder = './drive/My Drive/capstone/data/second_stage_stock/'
prediction_file = "./drive/My Drive/capstone/data/prediction/"
stock_file_path = prediction_file + 'predictions_dataset_stockmarket_Bloomberg.csv'
stock_classifier = TextClassifier.load(stock_data_folder + 'best-model.pt')
#benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')
stock_out_file_path = 'unannotated_stockmarket_Bloomberg_predictions.csv'

2020-05-28 23:47:23,307 loading file ./drive/My Drive/capstone/data/second_stage_stock/best-model.pt


In [0]:
finetuned_model_predictions(stock_file_path, stock_classifier, stock_out_file_path)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


All 177 rows done prediction! 
Done export!
